In [47]:
import lzma
from fuzzywuzzy import fuzz
import re
import os
import sys
from itertools import combinations, product
from collections import defaultdict
from tqdm.notebook import tqdm
from datetime import datetime


class keydefaultdict(defaultdict):
    def __missing__(self, key):
        if self.default_factory is None:
            raise KeyError(key)
        else:
            ret = self[key] = self.default_factory(key)
            return ret


compressed_length = keydefaultdict()

def compressor(quality):
    return (
        lambda s: len(lzma.compress(
            s,
            format=lzma.FORMAT_RAW,
            filters=[{"id": lzma.FILTER_LZMA2, "preset": quality}]
        ))
    )

def NCD(x, y, strategy=None):
    if min(len(x), len(y)) < 400:
        return 1.0
    if strategy == 'LEVENSHTEIN':
        return 1. - 0.01 * fuzz.ratio(x,y)
    
    global compressed_length

    lx = compressed_length[x]
    ly = compressed_length[y]

    if strategy == 'MINMAX':
        return (min(compressed_length[x + y], compressed_length[y + x]) - max(lx, ly)) / max(lx, ly)
    if strategy == 'METHOD-1':
        return (min(compressed_length[x + y], compressed_length[y + x]) - max(lx, ly)) / min(lx, ly)
    if strategy == 'METHOD-2':
        return (2*min(compressed_length[x + y], compressed_length[y + x]) - lx - ly) / (lx + ly)
    if strategy == 'METHOD-3':
        return min(
            (compressed_length[x + y] - lx) / ly,
            (compressed_length[y + x] - ly) / lx,
        )
        
    if strategy == 'FAST':
        if lx > ly:
            lx, ly = ly, lx
        return (compressed_length[y + x] - ly) / lx

    return (compressed_length[x + y] + compressed_length[y + x] - lx - ly) / (lx + ly)


In [48]:
rsol = re.compile(r'.*\\begin\{solution\}(.*)\\end\{solution\}.*', re.DOTALL)


def get_solution_text(filename):
    global rsol
    with open(filename, 'r', encoding='utf-8') as txt:
        solution_text = rsol.match(txt.read())
        if solution_text:
            return solution_text.group(1).strip().lower().replace(' ', '').encode()
        else:
            return None


def traverse(dirname, problem_ids=None):
    all_solutions = defaultdict(lambda: defaultdict(str))
    rexp = re.compile(r'[^1-9]*(\d+)\.tex')

    for item in os.listdir(dirname):
        if item.startswith('ds2021'):
            _, _, student_surname, student_name = item.split()

            for file in os.listdir(dirname + item):
                if file.endswith('.tex'):
                    x = rexp.match(file)
                    if x:
                        if not file.startswith('solution'):
                            print(item[13:] + '/' + file)

                        problem_id = int(x.group(1))
                        if problem_id < 2 or (
                                problem_ids is not None 
                                and problem_id not in problem_ids
                            ):
                            continue

                        solution_text = get_solution_text(dirname + item + '\\' + file)
                        if solution_text:
                            all_solutions[problem_id][f'{student_surname} {student_name}'] = solution_text
                        else:
                            print('Failed to find solution in ' + item[9:] + '/' + file)
    return all_solutions


def traverse_archive(dirname, year=None, problem_ids=None):
    old_solutions = defaultdict(lambda: defaultdict(str))
    rexp = re.compile(r'[^1-9]*(\d+)\.tex')

    for item in os.listdir(dirname):
        student_surname, student_name = item.split()

        for file in os.listdir(dirname + item):
            if file.endswith('.tex'):
                x = rexp.match(file)
                if x:
                    problem_id = int(x.group(1))
                    if problem_id < 2 or (
                                problem_ids is not None 
                                and problem_id not in problem_ids
                            ):
                        continue

                    solution_text = get_solution_text(dirname + item + '\\' + file)
                    if solution_text:
                        student_display = f'{student_surname} {student_name}'
                        if year:
                            student_display = f'{student_display} ({year})'
                        old_solutions[problem_id][student_display] = solution_text
                    else:
                        print('Failed to find solution in ' + item + '/' + file)


    return old_solutions

In [49]:
def find_plagiarism(threshold=0.5,
                    quality=9,
                    NCD_strategy=None,
                    problem_ids=None,
                    problem_ids_exclude=None,
                    exclusion_threshold=0.0,
                    check_archives=True, 
                    exclude_from_output=None,
                    focus_on_user=None
                   ):
    global compressed_length

    if exclude_from_output is None:
        exclude_from_output = dict()
        
    print('Loading this year solutions')
    all_solutions = traverse(r'D:\Dropbox\Apps\Overleaf\\')
    
    print('Loading last years solutions')
    old_solutions = defaultdict(lambda: defaultdict(str))
    
    if check_archives:
        archives = [
            traverse_archive(
                rf'd:\Documents\plagiarism-detection\ds\student-solutions-latex-{year}\\',
                year
            )
            for year in range(2015, 2019+1)
        ]
        for archive in archives:
            for problem_id in archive:
                old_solutions[problem_id].update(archive[problem_id])

    if compressed_length is None:
        compressed_length = keydefaultdict(compressor(quality))
    
    filtered_ids = set(all_solutions)
    if problem_ids:
        filtered_ids.intersection_update(problem_ids)
    if problem_ids_exclude is not None and not exclusion_threshold or exclusion_threshold <= 0:
        filtered_ids.difference_update(problem_ids_exclude)

    n_pairs_to_check = sum(k * (k - 1) // 2
                           for k in map(len, (all_solutions[problem_id]
                                              for problem_id in filtered_ids)))

    print('\nThis year solutions:', end='')
    similar_pairs = defaultdict(dict)

    with tqdm(total=n_pairs_to_check) as progress_bar:
        iteration = 0
        for problem_id in filtered_ids:
            if len(all_solutions[problem_id]) > 0:
                for (i1, s1), (i2, s2) in combinations(
                        all_solutions[problem_id].items(), 2):
                    if focus_on_user is not None and focus_on_user not in [i1, i2]:
                        continue
                    distance = NCD(s1, s2, NCD_strategy)
                    iteration += 1
                    progress_bar.update(1)
                    if distance <= exclusion_threshold or (
                            problem_ids_exclude is None or problem_id not in
                            problem_ids_exclude) and distance <= threshold:
                        similar_pairs[problem_id][(i1, i2)] = distance

    print('This year vs. last years solutions:', end='')
    n_pairs_to_check = sum(
        len(old_solutions[problem_id]) * len(all_solutions[problem_id])
        for problem_id in filtered_ids if problem_id in old_solutions)

    with tqdm(total=n_pairs_to_check) as progress_bar:
        for problem_id in filtered_ids:
            if problem_id in old_solutions:
                pairs_to_check = list(
                    product(all_solutions[problem_id].items(),
                            old_solutions[problem_id].items()))
                if len(pairs_to_check) > 0:
                    for (i1, s1), (i2, s2) in pairs_to_check:
                        if focus_on_user is not None and focus_on_user not in [i1, i2]:
                            continue
                        distance = NCD(s1, s2)
                        progress_bar.update(1)
                        if distance <= exclusion_threshold or (
                                problem_ids_exclude is None
                                or problem_id not in problem_ids_exclude
                        ) and distance <= threshold:
                            similar_pairs[problem_id][(i1, i2)] = distance

    sys.stdout.flush()
    print(f'Time of report: {datetime.now()}')
    no_duplicates = True
    if len(similar_pairs) > 0:
        for problem_id in sorted(similar_pairs.keys()):
            pairs_list = sorted([
                (round(similar_pairs[problem_id][pair], 2), pair[0], pair[1])
                for pair in similar_pairs[problem_id]
                if pair not in exclude_from_output.get(problem_id, {})
            ])
            if pairs_list != []:
                print(f'Similar solutions for problem {problem_id}:\n{pairs_list}')
                no_duplicates = False
    if no_duplicates:
        print('No duplicates')

In [50]:
exclude_from_output = {
    10: {('Мартьянов Вова', 'Шевляков Антон')},
    27: {('Дилшодзода Равшан', 'Мирзаев Рустам')},
    37: {('Волков Алексей', 'Инденбом Дмитрий (2019)'), 
         ('Ильдаров Адам', 'Пименов Павел (2016)'), 
         ('Ильдаров Адам', 'Левашов Артём (2018)')}
}

In [51]:
def find_plagiarism_single(problem_id,
                    solution_text,
                    threshold=0.5,
                    quality=9,
                    check_archives=True):
    global compressed_length
    solution_text = solution_text.strip().lower().replace(' ', '').encode()
    
    print('Loading this year solutions')
    all_solutions = traverse(r'D:\Dropbox\Apps\Overleaf\\', [problem_id])
    
    print('Loading last years solutions')
    old_solutions = defaultdict(lambda: defaultdict(str))
    
    if check_archives:
        archives = [
            traverse_archive(
                rf'd:\Documents\plagiarism-detection\ds\student-solutions-latex-{year}\\',
                year,
                [problem_id]
            )
            for year in range(2015, 2020+1)
        ]
        for archive in archives:
            for problem_id in archive:
                old_solutions[problem_id].update(archive[problem_id])

    all_solutions[problem_id].update(old_solutions[problem_id])
    compressed_length = keydefaultdict(compressor(quality))

    n_pairs_to_check = len(all_solutions[problem_id])

    print('\nThis year solutions:', end='')
    similar_pairs = defaultdict(dict)

    with tqdm(total=n_pairs_to_check) as progress_bar:
        iteration = 0
        if len(all_solutions[problem_id]) > 0:
            for (i2, s2) in all_solutions[problem_id].items():
                (i1, s1) = ('CURRENT', solution_text)
                distance = NCD(s1, s2)
                iteration += 1
                progress_bar.update(1)
                if distance <= threshold:
                    similar_pairs[problem_id][(i1, i2)] = distance

    sys.stdout.flush()
    no_duplicates = True
    if len(similar_pairs) > 0:
        for problem_id in sorted(similar_pairs.keys()):
            pairs_list = sorted([
                (round(similar_pairs[problem_id][pair], 2), pair[0], pair[1])
                for pair in similar_pairs[problem_id]
            ])

            if pairs_list != []:
                print(f'Similar solutions for problem {problem_id}:\n{pairs_list}')
                no_duplicates = False
    if no_duplicates:
        print('No duplicates')

In [52]:
import pickle

def store_cache(quality):
    global compressed_length

    with open(f'compressed_length_cache-q{quality}.pkl', 'wb') as outfile:
        pickle.dump({
            k: v for k,v in compressed_length.items()
        }, outfile)

def load_cache(quality):
    global compressed_length
    with open(rf'compressed_length_cache-q{quality}.pkl', 'rb') as infile:
        compressed_length_cache = pickle.load(infile)
        compressed_length = keydefaultdict(compressor(9))
        for k,v in compressed_length_cache.items():
            compressed_length[k] = v

In [60]:
load_cache(9)

find_plagiarism(
    threshold=0.5,
    quality=9,
#     NCD_strategy='METHOD-3',
#     problem_ids=[418],
    problem_ids_exclude=[],
    exclude_from_output=exclude_from_output,
    # focus_on_user='Коробко Максим'
)

Loading this year solutions
нко Михаил/main1.tex
Failed to find solution in Евтушенко Дмитрий/solution-441.tex
Failed to find solution in Кузнецов Николай/solution-4.tex
Failed to find solution in Кутный Данила/solution-276.tex
Failed to find solution in Лескин Иван/solution-5.tex
а Артём/Solution278.tex
Failed to find solution in Москалев Никита/solution-441.tex
Failed to find solution in Онучин Артем/solution-344.tex
Failed to find solution in Сенаторов Петр/solution-441.tex
Failed to find solution in Хайруллин Азат/solution-264.tex
Failed to find solution in Хайруллин Азат/solution-276.tex
Loading last years solutions

This year solutions:

  0%|          | 0/2205 [00:00<?, ?it/s]

This year vs. last years solutions:

  0%|          | 0/6560 [00:00<?, ?it/s]

Time of report: 2021-09-22 22:17:50.740358
Similar solutions for problem 278:
[(0.5, 'Зайченко Олеся', 'Павлунин Дмитрий')]


In [54]:
# solo = r''''''

In [55]:
# find_plagiarism_single(
#     problem_id=146,
#     solution_text=solo,
#     threshold=0.45,
#     quality=9,
#     check_archives=True
# )

In [56]:
len(compressed_length)

235184

In [58]:
# store_cache(9)